In [18]:
import pandas as pd
import numpy as np
from helpers import *
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.ticker import MaxNLocator
from geopy.geocoders import Nominatim
import folium
from folium.plugins import HeatMap
from IPython.display import display
from geopy.distance import geodesic
import math
from statsmodels.stats import diagnostic
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
beers = pd.read_csv("dataset/FilteredData/beers.csv")
breweries = pd.read_csv("dataset/FilteredData/breweries.csv")
# ratings = pd.read_csv("dataset/FilteredData/ratings.csv", dtype = {'user_id': str})
users = pd.read_csv("dataset/FilteredData/users.csv", dtype = {'user_id': str})
ratings = pd.read_csv('dataset/FilteredData/ratings.csv')

C:\Users\rishi\AppData\Local\Temp\ipykernel_33720\1270649451.py:5: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  ratings = pd.read_csv('dataset/FilteredData/ratings.csv')


In [25]:
us_state_populations = pd.read_csv('dataset/AdditionalData/US_state_populations.csv')
world_population = pd.read_csv("dataset/AdditionalData/world_population.csv")

#Contacenating the two datasets into one and creating a location-population dictionary:

us_state_populations.rename(columns = {'states': 'location'}, inplace = True)
populations_by_location = pd.concat([us_state_populations, world_population], ignore_index = True)
populations_by_location_dict = dict(zip(populations_by_location['location'], populations_by_location['2010']))

#Finding the portion (as percentage) of population for each location that are users in either of the websites. Then, we print the list of locations that are not present in the additional datasets. We see that all the locations are very small countries, so that exclusion of them will not be significant for our future analysis.

num_users_by_location = users['location'].value_counts().reset_index()
percentage_users_by_location = []
missing_locations = []
for index, row in num_users_by_location.iterrows():
    location = row['location']
    if location in populations_by_location_dict:
        percentage_users_by_location.append({'location': location, 'percentage': 100 * row['count'] / populations_by_location_dict[location], 'population': populations_by_location_dict[location], 'users': row['count']})
    else:
        missing_locations.append(location)

percentage_users_by_location = pd.DataFrame(percentage_users_by_location)

# Gets the percentage users by location


In [29]:
# Gets the latitude and longitude for each of our final locations in degrees
from geopy.geocoders import Nominatim
print(len(percentage_users_by_location['location'].tolist()))
latitudes_longitudes = {}
geolocator = Nominatim(user_agent="my_geocoder")
for location in percentage_users_by_location['location'].tolist():
    geocode_location = geolocator.geocode(location)
    if geocode_location:
        latitudes_longitudes[location] = (geocode_location.latitude, geocode_location.longitude)



71


In [33]:
import math

# Function taken from GeeksForGeeks
def haversine(lat1, lon1, lat2, lon2):
     
    
    dLat = (lat2 - lat1) * math.pi / 180.0 # Get distance between each of the lats and longs and convert to radians
    dLon = (lon2 - lon1) * math.pi / 180.0
 
    # convert to radians
    lat1 = (lat1) * math.pi / 180.0
    lat2 = (lat2) * math.pi / 180.0
 
    # apply formula
    a = (pow(math.sin(dLat / 2), 2) + pow(math.sin(dLon / 2), 2) * math.cos(lat1) * math.cos(lat2));
    RADIUS = 6371 # Approximate radius of the earth in kilometers
    c = 2 * math.asin(math.sqrt(a))
    return RADIUS * c

In [38]:
# Create a dictionary and dataframe of the distances between any two locations
location_distances_dict = {}
for first_location in latitudes_longitudes:
    location_distances_dict[first_location] = {}
    for second_location in latitudes_longitudes:
        if first_location == second_location:
            location_distances_dict[first_location][second_location] = 0
        if first_location != second_location:
            location_distances_dict[first_location][second_location] = haversine(latitudes_longitudes[first_location][0], latitudes_longitudes[first_location][1], latitudes_longitudes[second_location][0], latitudes_longitudes[second_location][1])

location_df = pd.DataFrame(location_distances_dict)
location_df

,"United States, California","United States, Pennsylvania","United States, New York","United States, Illinois","United States, Massachusetts","United States, Texas",Canada,"United States, Ohio","United States, Florida","United States, Michigan",...,"United States, Idaho","United States, Montana","United States, Alaska","United States, Hawaii","United States, North Dakota",Croatia,"United States, Wyoming",Northern Ireland,Iceland,Luxembourg
"United States, California",0.000000,3552.326937,3866.530542,2571.153846,4004.951531,1954.634315,2812.838572,3142.853522,3621.002712,2973.162506,...,870.428190,1403.352094,3702.099324,4032.957678,1922.491698,9809.535181,1192.887907,7968.884605,6825.690648,8996.221170
"United States, Pennsylvania",3552.326937,0.000000,314.369078,993.585872,498.189881,2150.884632,3025.700794,426.837717,1508.313085,643.204125,...,2974.961765,2622.285872,5134.699500,7583.233908,1951.333105,7138.588192,2462.128103,5256.793895,4550.129771,6283.974104
"United States, New York",3866.530542,314.369078,0.000000,1306.692620,247.576832,2433.926491,3220.710307,736.134907,1593.888355,936.620365,...,3280.879398,2915.022482,5335.381819,7897.019052,2238.177769,6909.851327,2770.694878,5041.720210,4400.658672,6058.185004
"United States, Illinois",2571.153846,993.585872,1306.692620,0.000000,1474.793721,1278.479719,2655.051895,573.396291,1553.439906,556.547317,...,2065.923217,1807.634735,4655.709614,6604.023708,1220.976690,7944.368427,1542.704390,6043.701727,5182.091791,7088.360485
"United States, Massachusetts",4004.951531,498.189881,247.576832,1474.793721,0.000000,2648.404725,3164.284000,921.058727,1835.719781,1036.934358,...,3379.745456,2984.625316,5275.993126,8025.753028,2301.717359,6667.541498,2879.906976,4796.463884,4155.345062,5815.078240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Croatia,9809.535181,7138.588192,6909.851327,7944.368427,6667.541498,9221.280638,7144.271882,7512.481458,8369.207604,7388.272921,...,8941.350196,8406.298231,7737.615519,12735.801456,7955.243341,0.000000,8685.090899,1901.909478,2990.526292,856.532972
"United States, Wyoming",1192.887907,2462.128103,2770.694878,1542.704390,2879.906976,1544.122668,1990.217949,2083.570878,2899.720603,1843.953268,...,523.322368,494.818458,3527.162354,5152.239161,738.358360,8685.090899,0.000000,6819.513340,5723.585890,7857.088851
Northern Ireland,7968.884605,5256.793895,5041.720210,6043.701727,4796.463884,7321.337309,5418.487290,5621.109484,6549.234384,5487.378655,...,7108.492110,6571.668023,6400.770662,11238.541541,6084.188573,1901.909478,6819.513340,0.000000,1309.384846,1048.906167
Iceland,6825.690648,4550.129771,4400.658672,5182.091791,4155.345062,6446.226737,4168.367187,4851.601963,5988.399710,4634.700053,...,5955.966443,5422.670297,5098.639920,9937.407490,5005.437309,2990.526292,5723.585890,1309.384846,0.000000,2216.835411


In [45]:
# Gets the five closest nearest locations for each location and the five furthest and stored in the dictionary
five_closest_locs = {}
five_farthest_locs = {}

for location in location_distances_dict:
    sorted_locations_by_distance = sorted(location_distances_dict[location].items(), key=lambda x:x[1])
    five_closest_locs[location] = sorted_locations_by_distance[1: 6]
    five_farthest_locs[location] = sorted_locations_by_distance[len(location_distances_dict) - 5 : len(location_distances_dict) ]
print(five_farthest_locs)
print(five_closest_locs)

{'United States, California': [('Poland', 9318.707523175612), ('Croatia', 9809.535181078592), ('Italy', 9914.873324007076), ('New Zealand', 11128.587578746901), ('Australia', 13031.883651960294)], 'United States, Pennsylvania': [('Italy', 7103.308940420564), ('Croatia', 7138.588191881768), ('United States, Hawaii', 7583.233907664657), ('New Zealand', 14289.608714493585), ('Australia', 16517.693140708885)], 'United States, New York': [('Italy', 6860.531870883095), ('Croatia', 6909.851327158158), ('United States, Hawaii', 7897.019052269386), ('New Zealand', 14561.6349722495), ('Australia', 16817.394454154168)], 'United States, Illinois': [('Poland', 7664.229381280188), ('Croatia', 7944.368426628417), ('Italy', 7945.3679179421815), ('New Zealand', 13370.852866960606), ('Australia', 15586.394660469059)], 'United States, Massachusetts': [('Italy', 6622.289858012425), ('Croatia', 6667.541497679188), ('United States, Hawaii', 8025.753027600149), ('New Zealand', 14785.524504377374), ('Australi

In [49]:
print(percentage_users_by_location['location']



['United States, California' 'United States, Pennsylvania'
 'United States, New York' 'United States, Illinois'
 'United States, Massachusetts' 'United States, Texas' 'Canada'
 'United States, Ohio' 'United States, Florida' 'United States, Michigan'
 'United States, New Jersey' 'United States, Virginia'
 'United States, North Carolina' 'United States, Washington'
 'United States, Minnesota' 'England' 'United States, Georgia'
 'United States, Indiana' 'United States, Wisconsin'
 'United States, Connecticut' 'United States, Colorado'
 'United States, Maryland' 'United States, Missouri'
 'United States, Oregon' 'Australia' 'United States, Alabama'
 'United States, Arizona' 'Poland' 'United States, Tennessee'
 'United States, South Carolina' 'United States, Kentucky' 'Netherlands'
 'United States, New Hampshire' 'United States, Iowa' 'Sweden' 'Germany'
 'Belgium' 'Denmark' 'Italy' 'Brazil' 'United States, Maine' 'Spain'
 'United States, Kansas' 'United States, Oklahoma'
 'United States, Ve